# 1.1 Amazon SageMaker Training 실습

## 학습 작업의 실행 노트북 개요

이 노트북은 SageMaker에서 Training Job을 통해서 모델 학습합니다.상세한 사항은 개발자 가이드를 참조 하세요. -->  [모델 학습](https://sagemaker.readthedocs.io/en/stable/overview.html#prepare-a-training-script)

- 일반적으로 SageMaker Training은 아래와 같이 진행이 됩니다.
    - [작업 실행 시 필요 라이브러리 import](#작업-실행-시-필요-라이브러리-import)
    - [SageMaker 세션과 Role, 사용 버킷 정의](#SageMaker-세션과-Role,-사용-버킷-정의)
    - [하이퍼파라미터 정의](#하이퍼파라미터-정의)
    - [학습 실행 작업 정의](#학습-실행-작업-정의)
        - 학습 코드 명
        - 학습 코드 폴더 명
        - 학습 코드가 사용한 Framework 종류, 버전 등
        - 학습 인스턴스 타입과 개수
        - SageMaker 세션
        - 학습 작업 하이퍼파라미터 정의
        - 학습 작업 산출물 관련 S3 버킷 설정 등
    - [학습 데이터셋 지정](#학습-데이터셋-지정)
        - 학습에 사용하는 데이터셋의 S3 URI 지정
    - [학습 실행](#학습-실행)
    - [데이터 세트 설명](#데이터-세트-설명)

### 작업 실행 시 필요 라이브러리 import

In [1]:
import boto3
import sagemaker

### Bucket 정의

In [20]:
use_default_bucket = True

if use_default_bucket:
    s3_bucket = sagemaker.session.Session().default_bucket()
else:
    s3_bucket = '<Type your bucket name here>'

###  훈련 잡에 사용할 SageMaker Role 정의 
- 로컬 머신에서 사용할 경우에는 아래의 주석을 해제하시고, SageMaker Execution Role ARN 을 넣어 주세요.

In [ ]:
use_local_machine = False
# use_local_machine = True

if use_local_machine:
    role = sagemaker.get_execution_role()
else:
    # role = '<Type Role ARN here>'
    role = 'arn:aws:iam::057716757052:role/dt2gsmoon'    

### 로컬 모드 혹은 클라우드 모드에 따른 SageMaker 세션 정의

In [3]:
#use_local_mode = False
use_local_mode = True

if use_local_mode:
    instance_type = "local"
    # instance_type = "local_gpu"
    from sagemaker.local import LocalSession
    sagemaker_session = LocalSession()
    sagemaker_session.config = {'local': {'local_code': True}}
else:
    sagemaker_session = sagemaker.session.Session()
    instance_type = "ml.m5.large"
    



In [4]:
bucket = sagemaker_session.default_bucket()
code_location = f's3://{bucket}/xgboost/code'
output_path = f's3://{bucket}/xgboost/output'

### 하이퍼파라미터 정의

In [5]:
hyperparameters = {
       "scale_pos_weight" : "29",    
        "max_depth": "3",
        "eta": "0.2",
        "objective": "binary:logistic",
        "num_round": "100",
}

### 학습 실행 작업 정의

In [8]:
from sagemaker.xgboost.estimator import XGBoost

estimator = XGBoost(
    entry_point="xgboost_starter_script.py",
    source_dir='src',
    output_path=output_path,
    code_location=code_location,
    hyperparameters=hyperparameters,
    role=role,
    sagemaker_session=sagemaker_session,
    instance_count=instance_count,
    instance_type=instance_type,
    framework_version="1.3-1",
    max_run=max_run,
    use_spot_instances=use_spot_instances,  # spot instance 활용
    keep_alive_period_in_seconds=1800,    
    max_wait=max_wait,
)

### 학습 데이터셋을 FSX 파일 시스템으로 카피 

In [9]:
is_fsx_data = False
is_delete_fsx_data = False
local_data_path = "../data/dataset/"
fsx_path = '/fsx/fraud/'

import os

def run_os_command(cmd):
    return_value = os.system(cmd)
    print("return_value: \n", return_value)
    

def prepare_fsx_data(is_fsx_data, local_data_path, fsx_path):
    if is_fsx_data:
        print("fsx_path: ", fsx_path)
        cmd = f'sudo cp -r {local_data_path} {fsx_path}'
        run_os_command(cmd)
        cmd = f'cd {fsx_path} && pwd && ls -R'    
        run_os_command(cmd)
        inputs = f'file://{fsx_path}'
        
    return inputs

if is_delete_fsx_data:    
    cmd = f'cd {fsx_path} && sudo rm -rf dataset && ls -R'    
    run_os_command(cmd)
    


In [10]:
if instance_type in ['local', 'local_gpu']:
    if is_fsx_data:
        inputs = prepare_fsx_data(is_fsx_data, local_data_path, fsx_path)    
    else:
        from pathlib import Path
        file_path = f'file://{Path.cwd()}'
        inputs = file_path.split('lab_1_training')[0] + 'data/dataset/'
else:
    data_path=f's3://{bucket}/xgboost/dataset'
    !aws s3 sync ../data/dataset/ $data_path    
    inputs = data_path

print("input for fsx_path: ", inputs)                

upload: ../data/dataset/test.csv to s3://sagemaker-us-east-1-057716757052/xgboost/dataset/test.csv
upload: ../data/dataset/train.csv to s3://sagemaker-us-east-1-057716757052/xgboost/dataset/train.csv
input for fsx_path:  s3://sagemaker-us-east-1-057716757052/xgboost/dataset


### 학습 실행

In [11]:
estimator.fit(inputs = {'inputdata': inputs},
                  wait=False)

In [12]:
%%time 

estimator.logs()

2023-05-07 10:31:48 Starting - Starting the training job...
2023-05-07 10:32:12 Starting - Preparing the instances for trainingProfilerReport-1683455508: InProgress
......
2023-05-07 10:33:12 Downloading - Downloading input data...
2023-05-07 10:33:53 Training - Downloading the training image...
2023-05-07 10:34:33 Uploading - Uploading generated training model[2023-05-07 10:34:17.832 ip-10-2-93-188.ec2.internal:7 INFO utils.py:28] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2023-05-07 10:34:17.857 ip-10-2-93-188.ec2.internal:7 INFO profiler_config_parser.py:111] User has disabled profiler.
[2023-05-07:10:34:17:INFO] Imported framework sagemaker_xgboost_container.training
[2023-05-07:10:34:17:INFO] No GPUs detected (normal if no gpus installed)
[2023-05-07:10:34:17:INFO] Invoking user training script.
[2023-05-07:10:34:18:INFO] Module xgboost_starter_script does not provide a setup.py. 
Generating setup.py
[2023-05-07:10:34:18:INFO] Generating setup.cfg
[2023-05-07:10:34:18:INFO] Generating M

### Second Training Job using Warm Pool

In [13]:
hyperparameters = {
       "scale_pos_weight" : "29",    
#        "max_depth": "3",
        "max_depth": "5",    
        "eta": "0.2",
        "objective": "binary:logistic",
        "num_round": "100",    
}

In [14]:
from sagemaker.xgboost.estimator import XGBoost

estimator = XGBoost(
    entry_point="xgboost_starter_script.py",
    source_dir='src',
    output_path=output_path,
    code_location=code_location,
    hyperparameters=hyperparameters,
    role=role,
    sagemaker_session=sagemaker_session,
    instance_count=instance_count,
    instance_type=instance_type,
    framework_version="1.3-1",
    max_run=max_run,
    use_spot_instances=use_spot_instances,  # spot instance 활용
    keep_alive_period_in_seconds=1800,    
    max_wait=max_wait,
)

In [15]:
estimator.fit(inputs = {'inputdata': inputs},
                  wait=False)

In [16]:
%%time 

estimator.logs()

2023-05-07 11:49:53 Starting - Starting the training job...
2023-05-07 11:50:15 Starting - Preparing the instances for trainingProfilerReport-1683460192: InProgress
......
2023-05-07 11:51:16 Downloading - Downloading input data...
2023-05-07 11:51:56 Training - Downloading the training image...
2023-05-07 11:52:36 Uploading - Uploading generated training model[2023-05-07 11:52:22.930 ip-10-2-246-70.ec2.internal:7 INFO utils.py:28] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2023-05-07 11:52:22.962 ip-10-2-246-70.ec2.internal:7 INFO profiler_config_parser.py:111] User has disabled profiler.
[2023-05-07:11:52:22:INFO] Imported framework sagemaker_xgboost_container.training
[2023-05-07:11:52:22:INFO] No GPUs detected (normal if no gpus installed)
[2023-05-07:11:52:22:INFO] Invoking user training script.
[2023-05-07:11:52:23:INFO] Module xgboost_starter_script does not provide a setup.py. 
Generating setup.py
[2023-05-07:11:52:23:INFO] Generating setup.cfg
[2023-05-07:11:52:23:INFO] Generating M

### 데이터 세트 설명
- 데이터 세트는 블로그 [Architect and build the full machine learning lifecycle with AWS: An end-to-end Amazon SageMaker demo](https://aws.amazon.com/blogs/machine-learning/architect-and-build-the-full-machine-learning-lifecycle-with-amazon-sagemaker/) 에서 사용한 데이터를 사용합니다. 블로그에서는 데이터 세트에 대해서 이렇게 설명 합니다.
- "자동차 보험 청구 사기를 탐지를 위해서 블로그 저자가 데이터를 합성해서 만든 고객과 클레임의 데이터 세트를 사용합니다."

In [17]:
import pandas as pd

In [18]:
train_prep_df = pd.read_csv('../data/dataset/train.csv')
train_prep_df.groupby('fraud').sample(n=5)

,fraud,vehicle_claim,total_claim_amount,customer_age,months_as_customer,num_claims_past_year,num_insurers_past_5_years,policy_deductable,policy_annual_premium,customer_zip,...,collision_type_missing,incident_severity_Major,incident_severity_Minor,incident_severity_Totaled,authorities_contacted_Ambulance,authorities_contacted_Fire,authorities_contacted_None,authorities_contacted_Police,police_report_available_No,police_report_available_Yes
1971,0,43996.793029,50196.793029,26,28,0,2,750,2650,85296,...,0,0,0,1,0,0,0,1,0,1
1623,0,1000.000000,4400.000000,59,212,0,1,750,2800,85735,...,1,0,1,0,0,0,0,1,1,0
580,0,2395.796502,91495.796502,42,179,0,1,750,2900,91605,...,1,1,0,0,0,0,0,1,1,0
3248,0,18757.696094,36457.696094,49,129,0,1,750,3000,98310,...,0,0,1,0,0,0,0,1,0,1
143,0,28265.784249,34265.784249,50,52,0,1,750,3000,90291,...,0,0,0,1,0,0,0,1,1,0
1335,1,14850.041265,31550.041265,40,45,0,2,750,3000,90302,...,0,1,0,0,0,0,0,1,1,0
2269,1,8158.963995,76458.963995,40,132,0,1,750,2750,95687,...,0,0,1,0,0,0,0,1,0,1
2178,1,8550.257202,31550.257202,20,9,0,4,750,2750,92567,...,0,0,1,0,0,0,1,0,1,0
38,1,14423.750237,57223.750237,50,24,0,4,750,3000,86404,...,0,1,0,0,0,0,0,1,1,0
2137,1,15720.265274,34020.265274,40,161,0,1,750,2700,85250,...,1,0,1,0,0,0,0,1,0,1


In [19]:
train_prep_df.groupby('fraud').size()

fraud
0    3869
1     131
dtype: int64

#### 데이터 컬럼 설명
- fraud: 보험 청구의 사기 여부 입니다. 1 이면 사기, 0 이면 정상 청구 입니다.
- vehicle_claim: 자동차에 대한 보험 청구액. 값으로서, $1000, $17,638 등이 있습니다.
- total_claim_amount: 전체 보험 청구액 입니다. $21,400, $10,000 등이 있습니다.    
- customer_age: 고객의 나이를 의미 합니다.
- months_as_customer: 고객으로서의 가입 기간을 의미합니다. 단위는 월로서 11, 30, 31 등의 값이 존재 합니다.
- num_claims_past_year: 작년의 보험 청구 수를 의미 합니다. 0, 1, 2, 3, 4, 5, 6 의 값이 존재 합니다.
- num_insurers_past_5_years: 과거 5년 동안의 보험 가입 회사 수를 의미 합니다. 1, 2, 3, 4, 5 의 값이 존재 합니다.
- policy_deductable: 보험의 최소 자기 부담금 입니다. $750, $800 등이 있습니다.    
- policy_annual_premium: 보험의 특약 가입에 대한 금액 입니다. $2000, $3000 등이 있습니다.
- customer_zip: 고객의 집 주소 우편 번호를 의미합니다.
- auto_year: 자동차의 년식을 의미 합니다. 2020, 2019 등이 있습니다.
- num_vehicles_involved: 몇 대의 자동차가 사고에 연관 되었는지 입니다. 1, 2, 3, 4, 5, 6 의 값이 있습니다.
- num_injuries: 몇 명이 상해를 입었는지를 기술합니다. 0, 1, 2, 3, 4, 의 값이 있습니다.
- num_witnesses: 몇 명의 목격자가 있었는지를 기술합니다. 0, 1, 2, 3, 4, 5 의 값이 있습니다.
- injury_claim: 상해에 대한 보험 청구액. \$5,500, \$70,700, \$100,700 등이 있습니다.    
- incident_month: 사고가 발생한 월을 의미합니다. 1~12 값이 존재 합니다.
- incident_day: 사고가 발생한 일자를 의미합니다. 1~31 값이 존재 합니다.
- incident_dow: 사고가 발생한 요일을 의미합니다. 0~6 값이 존재 합니다.
- incident_hour: 사고가 발생한 시간을 의미합니다. 0~23 값이 존재 합니다.
- policy_state: 보험 계약을 한 미국 주(State)를 의미 합니다. CA, WA, AZ, OR, NV, ID 가 존재 합니다.    
- policy_liability: 보험 청구의 한도를 의미 합니다. 예를 들어서 25/50 은  사람 당 상해 한도 $25,000, 사고 당 상해 한도가 $50,000 을  의미합니다. 25/50, 15/30, 30/60, 100/200 의 값이 존재 합니다. 
- customer_gender: 고객의 성별을 의미 합니다. Male, Female, Unkown, Other가 존재 합니다.
- customer_education: 고객의 최종 학력을 의미합니다. Bachelor, High School, Advanced Degree, Associate, Below High School 이 존재 합니다.
- driver_relationship: 보험 계약자와 운전자와의 관계 입니다. Self, Spouse, Child, Other 값이 존재 합니다.
- incident_type: 사고의 종류를 기술합니다. Collision, Break-in, Theft 값이 존재 합니다.
- collision_type: 충돌 타입을 기술합니다. Front, Rear, Side, missing 값이 존재 합니다.
- incident_severity: 사고의 손실 정도 입니다. Minor, Major, Totaled 값이 존재 합니다.
- authorities_contacted: 어떤 관련 기관에 연락을 했는지 입니다. Police, Ambuylance, Fire, None 값이 존재 합니다.
- police_report_available: 경찰 보고서가 존재하는지를 기술합니다. Yes, No 의 값이 있습니다.